# Tutorial 5: Preventing Undesirable Outputs from a Customer Service Chatbot

• Find  [the Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2007%20-%20Guarding_Against_Undesirable_Outputs_with_the_Self_Critique_Chain_Example.ipynb)  for the real-world example at  [towardsai.net/book](http://towardsai.net/book).

In this section, we will build a chatbot that can handle inquiries based on the content available on a website, such as blogs or documentation. The goal is to ensure the chatbot’s responses are appropriate and do not harm the brand’s reputation. This is particularly important when the chatbot may not find answers from its primary database.

The process begins with selecting webpages that can serve as the information source (in this case, using LangChain’s documentation pages). The content from these pages is stored in the Deep Lake vector database, facilitating retrieval.

First, use the `newspaper` library to extract content from each URL specified in the documents variable. Next, employ a recursive text splitter to segment the content into chunks of 1,000 characters, with a 100-character overlap between each segment.


In [ ]:
import newspaper
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = [
    'https://python.langchain.com/docs/get_started/introduction',
    'https://python.langchain.com/docs/get_started/quickstart',
    'https://python.langchain.com/docs/modules/model_io/models',
    'https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates'
]

pages_content = []

# Retrieve the Content
for url in documents:
    try:
        article = newspaper.Article( url )
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append({ "url": url, "text": article.text })
    except:
        continue

# Split to Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts, all_metadatas = [], []
for document in pages_content:
    chunks = text_splitter.split_text(document["text"])
    for chunk in chunks:
        all_texts.append(chunk)
        all_metadatas.append({ "source": document["url"] })

Initialize the `DeepLake` class, process records with an embedding function such as `OpenAIEmbeddings`, and store the data in the cloud using the `.add_texts()` method.

First, add the `ACTIVELOOP_TOKEN` key to your environment variables as follows.

In [ ]:
import os
from langchain_custom_utils.helper import get_activeloop_api_key
ACTIVELOOP_API_KEY = get_activeloop_api_key()

Then, execute the subsequent code snippet.

In [ ]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVELOOP_API_KEY
my_activeloop_dataset_name = "langchain_course_constitutional_chain"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)
db.add_texts(all_texts, all_metadatas)